In [1]:
import veryfi
import camelot
import pandas as pd
import pprint

In [2]:
from PyPDF2 import PdfFileReader, PdfFileWriter

pdf_file_path = 'doc.pdf'
file_base_name = pdf_file_path.replace('.pdf', '')

pdf = PdfFileReader(pdf_file_path)

pages = [0] 
pdfWriter = PdfFileWriter()

for page_num in pages:
    pdfWriter.addPage(pdf.getPage(page_num))

with open('{0}_subset.pdf'.format(file_base_name), 'wb') as f:
    pdfWriter.write(f)
    f.close()

In [4]:
client_id="vrfaycC0DtKLox02VEM32rvFxs79ikExMsMh4d4"
client_secret="lCLkVdsIgGRhcyWLdv4hCzvrdASepRO2WIVCMz4BhaoUzKvkRef5SniDYZXWXXlCGaJa2JtkhWH7VpJta6xHXYHJGDVbjrRkuOkCBWRc8td31Vh2udDkkOVwu4wpcLyk"
api_key="c7ffb7d1fa6c4e8e953f3bbf0f0befa3"
username="srishtisingh089"

In [5]:
client=veryfi.Client(client_id,client_secret,username,api_key)

In [6]:
json_result = client.process_document("doc_subset.pdf",categories=["Insurance"])

In [7]:
pprint.pprint(json_result)

{'abn_number': '',
 'account_number': '',
 'bill_to_address': 'BORIVALI W',
 'bill_to_name': 'KARUNA HOSPITALS',
 'bill_to_vat_number': '',
 'card_number': '0401',
 'cashback': 0.0,
 'category': '',
 'created': '2021-07-06 17:28:53',
 'currency_code': 'INR',
 'date': '2020-09-01 03:48:00',
 'delivery_date': '01/09/2020',
 'discount': 0.0,
 'document_reference_number': '166870662100000401',
 'document_title': '',
 'document_type': '',
 'due_date': '',
 'duplicate_of': 37531235,
 'external_id': '',
 'id': 37548756,
 'img_file_name': '380128d9-e18c-407e-858a-2db80bb13d8f.pdf',
 'img_thumbnail_url': 'https://scdn.veryfi.com/receipts/380128d9-e18c-407e-858a-2db80bb13d8f_t.jpg?Expires=1625593433&Signature=aTk92zcI~Fozg8WuS2jDRh-zuBmNKGUmLFIk3b3s6sfBzO9wQoslMhD94Zmvnijhv97-ehkCVX5DyJykanHnWv9ql8TlZewIvdEJwc96JwpuC6fNW29KbS3OrAscWzNwRAE1h1KfOcE~zLzAs812FKHJktS48d4NicJgfjwQKf-AmcbNesUmXpSJakjK85IBasbd5OT5wqQ5kVqQrzczYjcfbSI0HlZgzYxsR~Vqub4M3XtIqMCUR7ZUvMt-Fyw2VyfbdiMAvGlYMDPeIg6fTLM~mQqTRI4hXj1

In [8]:
print(json_result["bill_to_name"])
print(json_result["date"])
print(json_result["invoice_number"])

KARUNA HOSPITALS
2020-09-01 03:48:00
2021/171100/0277003


In [9]:
# extract all the tables in the PDF file
tables = camelot.read_pdf('doc_subset.pdf')
# number of tables extracted
print("Total tables extracted:", tables.n)

Total tables extracted: 3


In [10]:
table1=tables[1].df
table1

,0,1
0,Patient Name\n : HARSHAD DOSHI,Age :\n70YEARS\nGender : Male
1,,Expected Date of Admission\n: 29/08/2020
2,Policy Number\n : P/171100/01/2021/006902,Expected Date of Discharge\n: 01/09/2020
3,Policy Period\n : 29-JUL-2020 - 28-JUL-2021,Estimated length of stay\n:
4,Room\n : SHARING/SEMI PRIVATE ROOM\nCategory\n...,


In [11]:
table1[['c1','c2','none']]=table1[0].str.split(":",expand=True)
table1[['c3','c4','None']]=table1[1].str.split(":",expand=True)
x=table1.drop([0,1,"none","None"],axis=1)
x

,c1,c2,c3,c4
0,Patient Name\n,HARSHAD DOSHI,Age,\n70YEARS\nGender
1,,None,Expected Date of Admission\n,29/08/2020
2,Policy Number\n,P/171100/01/2021/006902,Expected Date of Discharge\n,01/09/2020
3,Policy Period\n,29-JUL-2020 - 28-JUL-2021,Estimated length of stay\n,
4,Room\n,SHARING/SEMI PRIVATE ROOM\nCategory\nA/C\nEli...,,None


In [12]:
name=x.loc[0,"c2"]
date_of_adm=x.loc[1,"c4"]
date_of_dis=x.loc[2,"c4"]

In [13]:
output=pd.Series({"Hospital name":json_result["bill_to_name"],
                     "Patient name":name,
                     "Bill number":json_result["invoice_number"],
                     "Bill date":json_result["date"],
                     "Admission Date":date_of_adm,
                     "Discharge Date":date_of_dis})

In [14]:
output

Hospital name        KARUNA HOSPITALS
Patient name            HARSHAD DOSHI
Bill number       2021/171100/0277003
Bill date         2020-09-01 03:48:00
Admission Date             29/08/2020
Discharge Date             01/09/2020
dtype: object

In [15]:
output.to_csv("output.csv",header=False)